<a href="https://colab.research.google.com/github/alxmarqs/LLMtopics/blob/main/1_2_Transformer_Encoder_only_exercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 4 — Exercício: Transformer *Encoder-only* com Embeddings Semânticos

## Objetivo
Treinar um modelo Transformer *encoder-only* com auto-supervisão via **Masked Language Model (MLM)** e demonstrar que os embeddings produzidos capturam a semântica dos documentos.

## Roteiro
1. Carregar um conjunto de documentos (dataset IMDB-Genres)
2. Treinar um tokenizador WordLevel sobre os documentos
3. Treinar o modelo encoder-only com MLM
4. **Teste:** extrair embeddings de dois subconjuntos temáticos distintos (Sci-Fi e Romance) e visualizar a separação com **PCA** e **t-SNE**

---
### Características do Encoder-only
- ✅ Atenção **bidirecional** (vê o contexto inteiro, não só o passado)
- ✅ Inferência via **pooling** (não autoregression)
- ✅ Treinamento auto-supervisionado via **MLM** (mascarar e prever tokens)

## Dependências

In [1]:
!pip install datasets tokenizers scikit-learn -q

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import random
import re
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Dispositivo: {device}')

Dispositivo: cpu


---
## Passo 1 — Carregar o conjunto de documentos

Usamos o dataset **IMDB-Genres** (descrições curtas de filmes). Cada documento é uma sinopse de filme.

In [3]:
from datasets import load_dataset

ds = load_dataset("jquigl/imdb-genres")

def clean_ascii(text):
    """Remove caracteres não-ASCII e símbolos desnecessários."""
    text = text.encode("ascii", errors="ignore").decode()
    return re.sub(r"[^A-Za-z0-9 .,:;!?'\-]", "", text)

# Extrai e limpa as descrições
documentos = [clean_ascii(x["description"]) for x in ds["train"]]
documentos = [t.split(" - ")[0] for t in documentos]   # remove ano/metadados no final
documentos = [t for t in documentos if len(t) > 20]     # descarta frases muito curtas

print(f"Total de documentos: {len(documentos)}")
print("\nPrimeiros 5 documentos:")
for d in documentos[:5]:
    print(" •", d)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/54.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/238256 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/29809 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/29756 [00:00<?, ? examples/s]

Total de documentos: 237457

Primeiros 5 documentos:
 • Flaming Ears is a pop sci-fi lesbian fantasy feature set in the year 2700 in the fictive burned-out city of Asche. It follows the tangled lives of three women
 • In a small unnamed town, in year 2025, Krsto works for the Agency which offers hearings for other people's secrets, with warranty that those secrets shall never be spread. At one of the ...                See full summary
 • The legendary Gulliver returns to the Kingdom of Lilliput, but he is not the giant they remember.
 • Seminal silent historical film, the story features King Munja, ruler of Aranti, famed warrior and patron of the arts. Munja Sandow falls into the hands of his arch enemy Tailap, who ...                See full summary
 • A look at the scandalous love triangle between Victorian art critic John Ruskin Greg Wise, his teenage bride Euphemia Effie Gray Dakota Fanning, and Pre-Raphaelite painter John Everett Millais Tom Sturridge.


---
## Passo 2 — Treinar o tokenizador

Usamos um tokenizador **WordLevel** simples: cada palavra única do corpus se torna um token.

- Vocabulário de 2000 palavras (adequado para o corpus pequeno)
- Tokens especiais: `[PAD]`, `[UNK]`, `[BOS]`, `[EOS]`, `[MASK]`
  - `[MASK]` é essencial para o MLM

In [4]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()

trainer = WordLevelTrainer(
    vocab_size=2000,
    special_tokens=["[PAD]", "[UNK]", "[BOS]", "[EOS]", "[MASK]"]
)

tokenizer.train_from_iterator(documentos, trainer)

vocab_size      = tokenizer.get_vocab_size()
pad_token_id    = tokenizer.token_to_id("[PAD]")
mask_token_id   = tokenizer.token_to_id("[MASK]")
bos_token_id    = tokenizer.token_to_id("[BOS]")
eos_token_id    = tokenizer.token_to_id("[EOS]")

print(f"Tamanho do vocabulário: {vocab_size}")
print(f"IDs especiais — PAD:{pad_token_id} | UNK:{tokenizer.token_to_id('[UNK]')} | BOS:{bos_token_id} | EOS:{eos_token_id} | MASK:{mask_token_id}")

# Funções auxiliares
def encode(text):
    ids = tokenizer.encode("[BOS] " + text + " [EOS]").ids
    return torch.tensor(ids, dtype=torch.long)

def decode(ids):
    return tokenizer.decode(ids.tolist())

# Exemplo
exemplo = documentos[0]
ids_exemplo = encode(exemplo)
print(f"\nExemplo → '{exemplo[:60]}...'")
print(f"Tokens: {ids_exemplo.tolist()[:15]} ...")
print(f"Decodificado: {decode(ids_exemplo)}")

Tamanho do vocabulário: 2000
IDs especiais — PAD:0 | UNK:1 | BOS:2 | EOS:3 | MASK:4

Exemplo → 'Flaming Ears is a pop sci-fi lesbian fantasy feature set in ...'
Tokens: [2, 1, 1, 13, 7, 1, 1, 17, 1945, 1, 867, 712, 161, 12, 6] ...
Decodificado: is a - fi fantasy feature set in the year in the - out city of . It follows the lives of three women


---
## Passo 3 — Definição do modelo Transformer *Encoder-only*

Arquitetura:
- **Token Embedding** + **Positional Embedding** aprendidos
- **N camadas TransformerEncoderLayer** com atenção bidirecional (sem máscara causal!)
- **LM Head** linear para predição MLM (usado apenas no treino)
- **Mean pooling** dos estados ocultos para gerar o embedding semântico (inferência)

In [5]:
class EncoderOnlyTransformer(nn.Module):
    """Transformer Encoder-only para embeddings semânticos via MLM."""

    def __init__(self, vocab_size, d_model=128, n_heads=4, num_layers=3, max_len=64):
        super().__init__()
        self.max_len = max_len

        # Embeddings de token e de posição (ambos aprendidos)
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb   = nn.Embedding(max_len, d_model)

        # Pilha de camadas do Encoder (atenção bidirecional — sem máscara causal)
        layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=256,
            dropout=0.1,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(layer, num_layers=num_layers)

        # Cabeça de predição MLM (usada apenas no treinamento)
        self.lm_head = nn.Linear(d_model, vocab_size)

    def initial_hidden_state(self, x):
        """Soma embedding de token + embedding posicional."""
        B, T = x.shape
        pos  = torch.arange(T, device=x.device).unsqueeze(0)   # (1, T)
        return self.token_emb(x) + self.pos_emb(pos)            # (B, T, d_model)

    def forward(self, x, src_key_padding_mask=None):
        """Forward para treinamento MLM — retorna logits (B, T, vocab_size)."""
        h      = self.initial_hidden_state(x)
        out    = self.encoder(h, src_key_padding_mask=src_key_padding_mask)
        logits = self.lm_head(out)
        return logits


# Instancia o modelo
model     = EncoderOnlyTransformer(vocab_size).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

total_params = sum(p.numel() for p in model.parameters())
print(f"Parâmetros totais: {total_params:,}")

Parâmetros totais: 919,632


---
## Passo 4 — Funções de treinamento MLM

### Como o MLM funciona
1. Seleciona aleatoriamente ~15% dos tokens de cada sequência
2. Substitui esses tokens por `[MASK]`
3. O modelo deve **prever os tokens originais** nas posições mascaradas
4. A loss é calculada apenas nas posições mascaradas (`ignore_index=-100`)

Isso força o encoder a aprender representações contextuais ricas, pois precisa usar o contexto bidirecional para adivinhar os tokens mascarados.

In [6]:
MAX_LEN = 32   # comprimento máximo das sequências no treino

def sample_batch(batch_size=32, max_len=MAX_LEN):
    """Amostra um batch de documentos, tokeniza e faz padding."""
    batch     = random.sample(documentos, batch_size)
    tokenized = [encode(t) for t in batch]

    max_t = min(max(len(x) for x in tokenized), max_len)
    padded = []
    for x in tokenized:
        x       = x[:max_t]
        pad_len = max_t - len(x)
        if pad_len > 0:
            x = torch.cat([x, torch.full((pad_len,), pad_token_id, dtype=torch.long)])
        padded.append(x)

    return torch.stack(padded)   # (B, T)


def apply_mlm_mask(batch, mask_prob=0.15):
    """
    Aplica máscara MLM:
    - 15% dos tokens são substituídos por [MASK]
    - Labels = token original nas posições mascaradas, -100 no resto
    """
    inputs = batch.clone()
    labels = batch.clone()

    probs      = torch.rand(batch.shape, device=batch.device)
    is_special = (batch == pad_token_id) | (batch == bos_token_id) | (batch == eos_token_id)
    mask_idx   = (probs < mask_prob) & (~is_special)

    inputs[mask_idx]  = mask_token_id   # substitui pelo token [MASK]
    labels[~mask_idx] = -100            # loss apenas nas posições mascaradas

    # Padding mask para o TransformerEncoder (True = ignorar posição)
    padding_mask = (inputs == pad_token_id)

    return inputs, labels, padding_mask


def get_semantic_embedding(text, model, max_len=MAX_LEN):
    """
    Extrai o embedding semântico de um texto:
    - Tokeniza e codifica
    - Passa pelo encoder (sem usar o lm_head)
    - Aplica mean pooling nos estados da sequência
    Retorna numpy array de shape (d_model,)
    """
    model.eval()
    with torch.no_grad():
        x = encode(text).to(device)
        x = x[:max_len].unsqueeze(0)        # (1, T)

        h   = model.initial_hidden_state(x) # (1, T, d_model)
        out = model.encoder(h)              # (1, T, d_model) — sem lm_head!

        # Mean pooling: média sobre todos os tokens da sequência
        embedding = out.mean(dim=1)         # (1, d_model)

    return embedding.squeeze(0).detach().cpu().numpy()  # (d_model,)

---
## Passo 5 — Treinamento MLM

In [ ]:
STEPS       = 5000
LOG_EVERY   = 500
loss_history = []

print(f"Treinando por {STEPS} steps com MLM...\n")

for step in range(1, STEPS + 1):
    model.train()

    batch                  = sample_batch(batch_size=32).to(device)
    inputs, labels, p_mask = apply_mlm_mask(batch)

    logits = model(inputs, src_key_padding_mask=p_mask)   # (B, T, V)

    loss = F.cross_entropy(
        logits.reshape(-1, vocab_size),
        labels.reshape(-1),
        ignore_index=-100
    )

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()

    loss_history.append(loss.item())

    if step % LOG_EVERY == 0:
        avg = np.mean(loss_history[-LOG_EVERY:])
        print(f"[step {step:>5}] loss = {avg:.4f}")

print("\nTreinamento concluído!")

Treinando por 5000 steps com MLM...



In [ ]:
# Curva de loss
window = 100
smooth = np.convolve(loss_history, np.ones(window)/window, mode='valid')

plt.figure(figsize=(9, 3))
plt.plot(loss_history, alpha=0.25, color='steelblue', label='Loss por step')
plt.plot(range(window-1, len(loss_history)), smooth, color='steelblue', lw=2, label=f'Média móvel ({window} steps)')
plt.xlabel('Step')
plt.ylabel('MLM Loss')
plt.title('Curva de Treinamento — MLM')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## Passo 6 — Teste: Visualização dos embeddings semânticos

### Preparação dos subconjuntos temáticos

Criamos dois subconjuntos com assuntos claramente distintos:
- 🚀 **Sci-Fi** — ficção científica, robôs, espaço, futuro
- 💕 **Romance** — amor, relacionamentos, emoções

Se o modelo aprendeu embeddings semânticos úteis, documentos do mesmo tema devem aparecer **agrupados** no espaço reduzido.

In [ ]:
# ---------------------------------------------------------------
# Subconjunto 1: Sci-Fi (ficção científica)
# ---------------------------------------------------------------
sci_fi_docs = [
    "A futuristic robot fights to save the galaxy from alien invaders.",
    "Space explorers discover a new planet with mysterious life forms.",
    "In the year 3000, artificial intelligence controls the world.",
    "A time traveler tries to prevent a nuclear war in the future.",
    "The spaceship crew encounters a black hole during their mission.",
    "Scientists engineer a new species to colonize Mars.",
    "A soldier is sent to the future to fight against machines.",
    "An astronaut gets stranded on an alien world and must survive.",
    "A hacker discovers the world is a computer simulation.",
    "Robots gain consciousness and rebel against their human creators.",
]

# ---------------------------------------------------------------
# Subconjunto 2: Romance
# ---------------------------------------------------------------
romance_docs = [
    "A young couple falls in love during a summer vacation in Paris.",
    "She met her soulmate at a coffee shop on a rainy day.",
    "A romantic wedding ceremony takes place on a beautiful beach.",
    "Two high school sweethearts reunite after twenty years apart.",
    "He realized he was in love with his best friend all along.",
    "A widow finds unexpected love after moving to a small town.",
    "Their first kiss happened under the stars on a warm summer night.",
    "She left everything behind to be with the man she loved.",
    "A passionate love story unfolds between two rival dancers.",
    "He proposed to her at the place where they first met.",
]

all_docs = sci_fi_docs + romance_docs
labels   = ["Sci-Fi"]  * len(sci_fi_docs) + ["Romance"] * len(romance_docs)

print(f"Subconjunto Sci-Fi : {len(sci_fi_docs)} documentos")
print(f"Subconjunto Romance: {len(romance_docs)} documentos")
print(f"Total              : {len(all_docs)} documentos")

In [ ]:
# ---------------------------------------------------------------
# Extração dos embeddings semânticos
# ---------------------------------------------------------------
print("Extraindo embeddings...")
embeddings = np.array([get_semantic_embedding(doc, model) for doc in all_docs])
print(f"Shape dos embeddings: {embeddings.shape}  →  ({len(all_docs)} docs, {embeddings.shape[1]} dims)")

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Paleta e configurações visuais
COLOR_MAP = {"Sci-Fi": "#2196F3", "Romance": "#E91E63"}
MARKER_MAP = {"Sci-Fi": "o",        "Romance": "s"}

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle("Embeddings Semânticos — Encoder-only Transformer", fontsize=14, fontweight="bold")

# ---------------------------------------------------------------
# Gráfico 1: PCA
# ---------------------------------------------------------------
pca      = PCA(n_components=2, random_state=42)
X_pca    = pca.fit_transform(embeddings)
var_exp  = pca.explained_variance_ratio_

ax = axes[0]
for label in ["Sci-Fi", "Romance"]:
    idx = [i for i, l in enumerate(labels) if l == label]
    ax.scatter(
        X_pca[idx, 0], X_pca[idx, 1],
        c=COLOR_MAP[label],
        marker=MARKER_MAP[label],
        s=120, alpha=0.85,
        edgecolors='white', linewidths=0.8,
        label=label
    )
    # Adiciona números nos pontos
    for j, i in enumerate(idx):
        ax.annotate(str(j+1), (X_pca[i,0], X_pca[i,1]),
                    textcoords="offset points", xytext=(5,3),
                    fontsize=7, color=COLOR_MAP[label])

ax.set_title(f"PCA  (var. explicada: PC1={var_exp[0]:.1%}, PC2={var_exp[1]:.1%})", fontsize=11)
ax.set_xlabel("Componente Principal 1")
ax.set_ylabel("Componente Principal 2")
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# ---------------------------------------------------------------
# Gráfico 2: t-SNE
# ---------------------------------------------------------------
tsne  = TSNE(n_components=2, perplexity=5, random_state=42, max_iter=1000)
X_tsne = tsne.fit_transform(embeddings)

ax = axes[1]
for label in ["Sci-Fi", "Romance"]:
    idx = [i for i, l in enumerate(labels) if l == label]
    ax.scatter(
        X_tsne[idx, 0], X_tsne[idx, 1],
        c=COLOR_MAP[label],
        marker=MARKER_MAP[label],
        s=120, alpha=0.85,
        edgecolors='white', linewidths=0.8,
        label=label
    )
    for j, i in enumerate(idx):
        ax.annotate(str(j+1), (X_tsne[i,0], X_tsne[i,1]),
                    textcoords="offset points", xytext=(5,3),
                    fontsize=7, color=COLOR_MAP[label])

ax.set_title("t-SNE  (perplexidade=5)", fontsize=11)
ax.set_xlabel("Componente t-SNE 1")
ax.set_ylabel("Componente t-SNE 2")
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("embeddings_semanticos.png", dpi=150, bbox_inches='tight')
plt.show()
print("Figura salva em 'embeddings_semanticos.png'")

---
## Passo 7 — Análise quantitativa: distâncias intra vs inter cluster

Para confirmar numericamente a separação, calculamos:
- **Distância intra-cluster** (dentro do mesmo tema) → esperado: menor
- **Distância inter-cluster** (entre temas diferentes) → esperado: maior

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

idx_sf  = [i for i, l in enumerate(labels) if l == "Sci-Fi"]
idx_rom = [i for i, l in enumerate(labels) if l == "Romance"]

emb_sf  = embeddings[idx_sf]
emb_rom = embeddings[idx_rom]

# Distância coseno (0 = idêntico, 1 = ortogonal)
dist_intra_sf  = cosine_distances(emb_sf).mean()
dist_intra_rom = cosine_distances(emb_rom).mean()
dist_inter     = cosine_distances(emb_sf, emb_rom).mean()

print("=" * 45)
print("   Distâncias Coseno entre Embeddings")
print("=" * 45)
print(f"  Intra-cluster Sci-Fi  : {dist_intra_sf:.4f}")
print(f"  Intra-cluster Romance : {dist_intra_rom:.4f}")
print(f"  Inter-cluster         : {dist_inter:.4f}")
print("=" * 45)
ratio = dist_inter / ((dist_intra_sf + dist_intra_rom) / 2)
print(f"  Razão inter/intra     : {ratio:.2f}x")
print()
if ratio > 1.0:
    print("✅ Documentos do mesmo tema estão mais próximos entre si")
    print("   do que documentos de temas diferentes.")
    print("   O modelo aprendeu embeddings semânticos úteis!")
else:
    print("⚠️  A separação ainda não é clara — considere mais steps de treino.")

---
## Conclusões

### O que foi implementado

| Componente | Descrição |
|---|---|
| **Corpus** | 100 sinopses de filmes (IMDB-Genres) |
| **Tokenizador** | WordLevel com vocab=2000, treinado no corpus |
| **Arquitetura** | Transformer Encoder-only (3 camadas, 4 heads, d=128) |
| **Treinamento** | MLM auto-supervisionado (15% de tokens mascarados) |
| **Inferência** | Mean pooling dos estados ocultos → vetor semântico |
| **Avaliação** | PCA + t-SNE + distâncias coseno intra/inter-cluster |

### Por que o Encoder-only funciona assim?

- **Atenção bidirecional**: cada token vê o contexto à esquerda E à direita, ao contrário do decoder-only que só vê o passado. Isso gera representações mais ricas da semântica.
- **MLM como tarefa de pré-treino**: para adivinhar um token mascarado, o modelo precisa entender o contexto completo da frase, forçando-o a capturar significado.
- **Mean pooling**: ao fazer a média dos vetores de todos os tokens, comprimimos a sequência inteira em um único vetor que resume o conteúdo do documento.
- **Separação no espaço**: documentos sobre o mesmo tema usam vocabulário similar → o modelo aprende que esses contextos são parecidos → embeddings ficam próximos no espaço vetorial.